In [1]:
import src.lol_utils as lol
import pandas as pd

## Creating Matrix
We will shortly recreate the matrix that we created in our last notebook.

In [2]:
df = pd.read_pickle('cleaned_features.pkl')

for champ in lol.champion_list:
    df[f'{champ}_presence'] = df.apply(
        lambda row: int(champ in row['blue_team'] or champ in row['red_team']),
        axis=1
    )

champ_presence_cols = [f'{champ}_presence' for champ in lol.champion_list]
extra_cols = ['synergy_blue', 'synergy_red', 'counter_delta']
feature_cols = champ_presence_cols + [col for col in extra_cols if col in df.columns]

for col in feature_cols:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype(int)


X = df[feature_cols]
y = df['target']

C:\Users\chris\AppData\Local\Temp\ipykernel_6020\3901340183.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{champ}_presence'] = df.apply(
C:\Users\chris\AppData\Local\Temp\ipykernel_6020\3901340183.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{champ}_presence'] = df.apply(
C:\Users\chris\AppData\Local\Temp\ipykernel_6020\3901340183.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colu

We split the dataset into training and testing sets, for our model.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

For our models, we will consider LightGBM and XGBoost. We consider this model to
1. High-dimenensoinal features
2. High sparsity
3. Is good at capturing interactions between the different champions.
4. Good at reducing noisy winrates (especially on rare champions).

In [4]:
model = lol.lgb.LGBMClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

acc = lol.accuracy_score(y_test, y_pred)
print(f"LightGBM Test Accuracy: {acc:.4f}")

[LightGBM] [Info] Number of positive: 1736, number of negative: 1775
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002481 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1099
[LightGBM] [Info] Number of data points in the train set: 3511, number of used features: 170
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.494446 -> initscore=-0.022217
[LightGBM] [Info] Start training from score -0.022217
LightGBM Test Accuracy: 0.7733


In [5]:
# Train an XGBoost model instead of LightGBM

import xgboost as xgb
from sklearn.metrics import accuracy_score

# Create and train the XGBoost model
model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate accuracy
acc = accuracy_score(y_test, y_pred)
print(f"XGBoost Test Accuracy: {acc:.4f}")

XGBoost Test Accuracy: 0.7847


c:\Users\chris\anaconda3\envs\lol_match_prediction\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:31:57] WARNING: D:\bld\xgboost-split_1758007502304\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Since both models seem comparable and equivalent, we will use XGBoost. 

We will determine what are the most important features that indicate the easiest thing to predict. We also show the least important feature (excluding zero) to find what matters the least.

In [8]:
# ...existing code...
importances = model.feature_importances_
feature_names = X.columns

# Indices of non-zero importances
nz_idx = lol.np.where(importances > 0)[0]

if nz_idx.size == 0:
    print("No non-zero feature importances.")
else:
    # Top 10 (non-zero)
    top_order = nz_idx[lol.np.argsort(importances[nz_idx])[::-1]]
    top_indices = top_order[:10]
    print("Top 10 most important features (non-zero):")
    for i in top_indices:
        print(f"{feature_names[i]}: {importances[i]:.4f}")

    # Least 10 (smallest positive)
    print("\nLeast 10 important features (non-zero):")
    bottom_order = nz_idx[lol.np.argsort(importances[nz_idx])]
    bottom_indices = bottom_order[:10]
    for i in bottom_indices:
        print(f"{feature_names[i]}: {importances[i]:.4f}")
# ...existing code...

Top 10 most important features (non-zero):
Talon_presence: 0.0145
Urgot_presence: 0.0144
Tristana_presence: 0.0118
Leona_presence: 0.0115
Jinx_presence: 0.0114
Gnar_presence: 0.0112
Karthus_presence: 0.0110
Chogath_presence: 0.0110
Shen_presence: 0.0108
Elise_presence: 0.0106

Least 10 important features (non-zero):
Shyvana_presence: 0.0007
Xerath_presence: 0.0011
Heimerdinger_presence: 0.0011
Annie_presence: 0.0013
Lux_presence: 0.0015
Graves_presence: 0.0015
Kassadin_presence: 0.0018
Rammus_presence: 0.0019
Janna_presence: 0.0019
Nasus_presence: 0.0021


We can clearly see that champions that are typically not known to be meta in high-elo (Shyvana, Xerath, Heimerdinger, Annie, Rammus, Nasus, etc.) are shown to have the least impact. This clearly makes sense as all these champions are known to be much weaker in higher-tier of play due to their obvious weakness that higher-elo players are better at playing around.

## Parameterization
We will now do multiple runs of the model in order to see which parameter best suits our model

In [9]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    "n_estimators": [100, 300, 600, 1000],
    "max_depth": [3, 4, 6, 8, 10],
    "learning_rate": [0.01, 0.05, 0.1, 0.3, 0.7, 1],
    "subsample": [0.5, 0.7, 0.8, 1.0],
    "colsample_bytree": [0.5, 0.7, 0.8, 1.0],
    "gamma": [0, 1, 5],
    "reg_alpha": [0, 0.1, 1],
    "reg_lambda": [1, 1.5, 2]
}

xgb_clf = xgb.XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

search = RandomizedSearchCV(
    xgb_clf,
    param_distributions=param_dist,
    n_iter=30,  # Increase for more thorough search
    scoring='accuracy',
    cv=3,
    verbose=2,
    n_jobs=-1
)

search.fit(X_train, y_train)
print("Best parameters:", search.best_params_)
print("Best CV accuracy:", search.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


c:\Users\chris\anaconda3\envs\lol_match_prediction\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:43:37] WARNING: D:\bld\xgboost-split_1758007502304\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best parameters: {'subsample': 0.5, 'reg_lambda': 1.5, 'reg_alpha': 1, 'n_estimators': 600, 'max_depth': 8, 'learning_rate': 0.1, 'gamma': 0, 'colsample_bytree': 0.5}
Best CV accuracy: 0.7584709783685019


We can see the best parameters for this specific dataset can create an accuracy to 76%. Due to the fact that these games often contain 10 players, being able to predict 76% of games just purely based on draft is something I believe is very impressive. There lots of variables that we can not control, like the player themself (some players will defy this by being so much better than their competition) or a player not playing to the best of their capability (some players are autofilled and are playing champions they aren't used to playing).

Let's train the model using the best hyperparameters now and run some mock tests.

In [10]:
# Run a mock trial using the best hyperparameters from RandomizedSearchCV

# Get best parameters from search
best_params = search.best_params_

# Create a new XGBoost model with best parameters
mock_model = xgb.XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss',
    **best_params
)

# Train on the full training set
mock_model.fit(X_train, y_train)

# Select a random sample of 5 matches from the test set
mock_sample = X_test.sample(5, random_state=42)
mock_true = y_test.loc[mock_sample.index]
mock_pred = mock_model.predict(mock_sample)

print("Mock Trial Results:")
for i in range(len(mock_sample)):
    print(f"Match {i+1}:")
    print(f"Features: {mock_sample.iloc[i].to_dict()}")
    print(f"True Outcome: {mock_true.iloc[i]}")
    print(f"Predicted Outcome: {mock_pred[i]}")
    print("-" * 30)

c:\Users\chris\anaconda3\envs\lol_match_prediction\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:43:50] WARNING: D:\bld\xgboost-split_1758007502304\work\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Mock Trial Results:
Match 1:
Features: {'Aatrox_presence': 0.0, 'Ahri_presence': 0.0, 'Akali_presence': 0.0, 'Akshan_presence': 0.0, 'Alistar_presence': 0.0, 'Amumu_presence': 0.0, 'Anivia_presence': 0.0, 'Annie_presence': 0.0, 'Aphelios_presence': 0.0, 'Ashe_presence': 0.0, 'AurelionSol_presence': 0.0, 'Azir_presence': 0.0, 'Bard_presence': 0.0, 'Belveth_presence': 0.0, 'Blitzcrank_presence': 0.0, 'Brand_presence': 0.0, 'Braum_presence': 0.0, 'Briar_presence': 0.0, 'Caitlyn_presence': 0.0, 'Camille_presence': 0.0, 'Cassiopeia_presence': 0.0, 'Chogath_presence': 0.0, 'Corki_presence': 0.0, 'Darius_presence': 0.0, 'Diana_presence': 0.0, 'DrMundo_presence': 0.0, 'Draven_presence': 0.0, 'Ekko_presence': 0.0, 'Elise_presence': 0.0, 'Evelynn_presence': 0.0, 'Ezreal_presence': 0.0, 'Fiddlesticks_presence': 0.0, 'Fiora_presence': 0.0, 'Fizz_presence': 0.0, 'Galio_presence': 0.0, 'Gangplank_presence': 0.0, 'Garen_presence': 0.0, 'Gnar_presence': 0.0, 'Gragas_presence': 0.0, 'Graves_presence': 

We can even use this model to predict real-time games. To do so we can just change the champions on the respective team.

In [13]:
# Correct champion names for blue and red teams
blue_team = ['Vladimir', 'Reksai', 'Yasuo', 'Hwei', 'Rakan']
red_team = ['KSante', 'Rengar', 'Ryze', 'Tristana', 'Alistar']

# Calculate champion presence (side-agnostic)
mock_features = {}
for champ in lol.champion_list:
    mock_features[f'{champ}_presence'] = int(champ in blue_team or champ in red_team)

# Calculate synergy and counter features using your utility functions
# First, get champion winrates from your training data
champ_winrates = lol.calculate_champion_winrates(df)

# Use your synergy and counter methods for a single match
import pandas as pd
mock_match_df = pd.DataFrame([{
    'blue_team': blue_team,
    'red_team': red_team
}])

# Calculate synergy and counter scores for this match
synergy_blue, synergy_red = lol.calculate_team_synergy(mock_match_df, champ_winrates)
counter_delta = lol.calculate_team_counters(mock_match_df, champ_winrates)

mock_features['synergy_blue'] = synergy_blue.iloc[0]
mock_features['synergy_red'] = synergy_red.iloc[0]
mock_features['counter_delta'] = counter_delta.iloc[0]

# Create DataFrame for the mock match
mock_df = pd.DataFrame([mock_features])

# Predict outcome using the best model
mock_pred = mock_model.predict(mock_df)
print("Mock Team Combination:")
print("Blue team:", blue_team)
print("Red team:", red_team)
print(f"Synergy Blue: {mock_features['synergy_blue']:.3f}")
print(f"Synergy Red: {mock_features['synergy_red']:.3f}")
print(f"Counter Delta: {mock_features['counter_delta']:.3f}")
print(f"Predicted outcome (1=blue win, 0=red win): {mock_pred[0]}") 

Mock Team Combination:
Blue team: ['Vladimir', 'Reksai', 'Yasuo', 'Hwei', 'Rakan']
Red team: ['KSante', 'Rengar', 'Ryze', 'Tristana', 'Alistar']
Synergy Blue: 302.029
Synergy Red: 557.661
Counter Delta: -112.479
Predicted outcome (1=blue win, 0=red win): 0
